In [1]:
import json
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
train_path = 'SCOTUS/train.json'
dev_path = 'SCOTUS/dev.json'

In [3]:
with open(train_path, 'r', encoding="utf-8") as f:
    train = json.load(f)

In [4]:
def html_processing(html_content):
    """Récupère le contenu intéressant du html en gardant les balises a, em et blockquote"""
    
    soup = BeautifulSoup(html_content, 'html.parser')

    # On suppose que le contenu intéressant ne se trouve que dans cette classe
    div_class = "-display-inline-block text-left"
    target_div = soup.find('div', class_=div_class)

    # Extraire le texte du div en fusionnant les textes des balises <a> et <em> avec les textes précédent et suivant
    # et en gardant les balises <a>, <em> et <blockquote>
    if target_div:
        temp_result = []
        to_merge = False
        for element in target_div.children:
            if element.getText() == '\n':
                continue
            elif element.name == "a" or element.name == "em" :
                temp_result.append("<" + element.name + "> " + element.get_text() + " </" + element.name + ">")
                to_merge = True
            elif (to_merge) :
                if (temp_result[-1].startswith("<em>")):
                    temp_result[-2] = temp_result[-2] + temp_result[-1] + element.get_text()
                else:
                    temp_result[-2] = temp_result[-2] + " " + temp_result[-1] + " " + element.get_text()
                temp_result.pop()
                to_merge = False
            elif (element.name == "blockquote") :
                temp_result.append("<" + element.name + ">" + element.get_text() + "</" + element.name + ">")
            else :
                temp_result.append(element.get_text())

    else:
        print(f"Aucun div trouvé avec la classe '{div_class}'.")

    result = []

    for res in temp_result:
        if '\t' in res:
            string = res.replace('\t', '').replace('\n', ' ')
            result.append(string)
        else:
            if len(result) < 1:
                result.append(res.replace('\n', ''))
            else:
                result[-1] = result[-1] + " " + res.replace('\n', '')

    return result


In [5]:
for i in range(len(train)):
    html_content = train[i]["raw_source"]

    text = html_processing(html_content)

    name = 'data_txt_save/train_' + str(i) + '.txt'

    with open(name, 'w', encoding='utf-8') as f:
        for line in text:
            f.write(line + '\n')